In [ ]:
import sys
sys.path.append('../')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
import torch
import pandas as pd

from utils.utils_data import get_camelyon_test_dataloader, get_wssb_test_dataloader
from models.DVBCDModel import DVBCDModel

print(torch.__version__)
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
torch.autograd.set_detect_anomaly(True)

USE_GPU = True
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
if USE_GPU and torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('using device:', DEVICE)

In [ ]:
MAIN_PATH = "/work/work_fran/Deep_Var_BCD/"
SAVE_MODEL_PATH = MAIN_PATH + "weights/0pe_224ps_0.25theta_0.05sigmaRui_60000nsamples/"

N_SAMPLES = 60000
PATCH_SIZE = 224
NUM_WORKERS = 64
WSSB_DATA_PATH = "/data/BasesDeDatos/Alsubaie/Data/"
CAMELYON_DATA_PATH = "/data/BasesDeDatos/Camelyon/Camelyon17/training/patches_224/"

In [ ]:
model = DVBCDModel(device=DEVICE)
model.load(SAVE_MODEL_PATH + "best.pt", remove_module=False)

In [ ]:
test_dataloader_wssb_dict = get_wssb_test_dataloader(WSSB_DATA_PATH, NUM_WORKERS)
results_dic_wssb = {}
for organ in test_dataloader_wssb_dict.keys():
    metrics_wssb = model.evaluate_GT(test_dataloader_wssb_dict[organ])
    for k in metrics_wssb.keys():
        results_dic_wssb[f"wssb_{organ}_" + k] = metrics_wssb[k]
res_wssb_df = pd.DataFrame(results_dic_wssb, index=[0])
res_wssb_df

In [ ]:
#n_samples_camelyon = int(0.2*N_SAMPLES)
#test_dataloader_camelyon = get_camelyon_test_dataloader(CAMELYON_DATA_PATH, PATCH_SIZE, NUM_WORKERS, n_samples_camelyon)
#results_dic_camelyon = {}
#metrics_camelyon = model.evaluate(test_dataloader_camelyon)
#for k in metrics_camelyon.keys():
#    results_dic_camelyon["camelyon_" + k] = metrics_camelyon[k]
#res_camelyon_df = pd.DataFrame(results_dic_camelyon, index=[0])
#res_camelyon_df